This is a demo of how to use 'rok_estimator.py'.

In [1]:
from rok_estimator import *
from lattice_lib import *
from lattice_lib.util import *

In [2]:
# Specify parameters related to the underlying ring
ring_60 = RingParam(f=60)

# Create a new starting relation
rel = Relation(ring=ring_60,wdim=60,log_beta_wit_inf=0)
rel.show()

# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp, cost_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show(brief=True)
cost_bdecomp.show()

# Running pi_split reducing rel into rel_split
rel_split, cost_split = rel.pi_split(2)
rel_split.show(brief=True)
cost_split.show()

# Running pi_batch reducing rel into rel_batch
rel_batch, cost_batch = rel.pi_batch()
rel_batch.show(brief=True)
cost_batch.show()

# Running pi_fold reducing rel into rel_fold
rel_fold, cost_fold = rel.pi_fold(23)
rel_fold.show(brief=True)
cost_fold.show()

# Running pi_norm reducing rel into rel_norm
rel_norm, cost_norm = rel.pi_norm()
rel_norm.show(brief=True)
cost_norm.show()

Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 41, n_commit = 40, n_rel = 1
    wdim = 60, rep = 1
    log_2-norm (real | extr) = (8  | 0 ), log_inf-norm (real | extr) = (0 | 0)
    wit size = 120.0 B 
 
Relation: wdim =       60, rep =   2, log_2-norm (real | extr) = (  8  |   0 ), log_inf-norm (real | extr) = (  0 |   0), wit size = 240.0 B 
Cost: communication = 5.125 KB, soundness error = 2^-Infinity
 
Relation: wdim =       30, rep =   2, log_2-norm (real | extr) = (  8  |   0 ), log_inf-norm (real | extr) = (  0 |   0), wit size = 120.0 B 
Cost: communication = 5.375 KB, soundness error = 2^-128
 
Relation: wdim =       60, rep =   1, log_2-norm (real | extr) = (  8  |   0 ), log_inf-norm (real | extr) = (  0 |   0), wit size = 120.0 B 
Cost: communication = 0

In [7]:
# Simulate the execution of a sequence of RoKs.
# In the following example, the underlying ring is chosen to be the 60-th cyclotomic ring. 
# The SIS norm bound is 51-bit and the modulus is 64-bit. 
# The initial relation is a 2**21-by-2**5 matrix of binary ring elements which can be represented in 128 MB.
# The canonical ell_2-norms of the initial and extracted witnesses are both 2**15. 
# The estimated proof size is 1.959 MB.

ring_params = {
    "f": 7,
    "log_beta_sis_2": 43,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**21,
    "rep": 2**5,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "repout": rel_params["rep"],
    "ell": 2,
    "d": 4,
}

opener = [("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {"repout": ops_params["repout"]})]
loop = [("bdecomp", {"ell": ops_params["ell"]}), ("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {"repout": ops_params["repout"]})]
ops = opener + 4 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

Ring parameters:
    conductor f: 7, degree phi: 6, modulus q: 2^64, beta_sis_2: 2^43
    SIS module rank n_sis: 188, SIS security secpar: 128
    residue degree: 2, Schwartz-Zippel security kappa: 128
    |R_q| = 48.00 B, |R_q^(n_sis)| = 8.812 KB
 
Subtractive set parameters:
    cardinality: 7
    forward ell_2 expansion factor gamma_2: 2^3
    inverse ell_2 expansion factor theta_2: 2^2
    forward ell_inf expansion factor gamma_inf: 2^3
    inverse ell_inf expansion factor theta_inf: 2^3
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 189, n_commit = 188, n_rel = 1
    wdim = 2097152, rep = 32
    log_2-norm (real | extr) = (14  | 14 ), log_inf-norm (real | extr) = (0 | 14)
    wit size = 48.00 MB
 
Execution Trace:
init    : wdim =  2097152, rep =  32, log_2-n